In [9]:
import os
import torch
import glob
from tqdm import tqdm

from torchmetrics.detection.mean_ap import MeanAveragePrecision
from ultralytics import YOLO
from sahi.predict import get_sliced_prediction
from sahi import AutoDetectionModel
import matplotlib.pyplot as plt
import numpy as np
from sahi.utils.cv import read_image

In [10]:
DATASET_YAML = './Heridal_patched/data.yaml'
IMG_SIZE = 640
BATCH = 8
EPOCHS = 300
PATIENCE = 30
PROJECT = 'runs'
RUN_NAME = 'yolov8s_patched'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

FULL_IMG_DIR = './Heridal_patched/test/images'
FULL_LABEL_DIR = './Heridal_patched/test/labels'
CONF_THRESHOLD = 0.5
SLICE_OVERLAP = 0.1
MODEL_PATH = f'./{PROJECT}/{RUN_NAME}/weights/best.pt'

In [11]:
def train_model():
    torch.cuda.empty_cache() 
    model = YOLO('/mnt/c/Study/Python/DroneDetection/Heridal_patched/yolov8s.pt')

    model.train(
        data=DATASET_YAML,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH,
        device=DEVICE,
        name=RUN_NAME,
        project=PROJECT,
        workers=2,
        patience = PATIENCE,
        single_cls=True,
    )
    return model

In [12]:
def load_gt_yolo(label_path, img_w, img_h):
    boxes = []
    if not os.path.exists(label_path): return boxes
    with open(label_path) as f:
        for line in f:
            cls, x, y, w, h = map(float, line.strip().split())
            cx, cy = x * img_w, y * img_h
            bw, bh = w * img_w, h * img_h
            x1, y1 = cx - bw/2, cy - bh/2
            x2, y2 = cx + bw/2, cy + bh/2
            boxes.append([x1, y1, x2, y2])
    return boxes

In [ ]:
def evaluate_map50(model_path):
    model = AutoDetectionModel.from_pretrained(
        model_type="yolov8",
        model_path=model_path,
        confidence_threshold=CONF_THRESHOLD,
        device=DEVICE
    )

    metric = MeanAveragePrecision()

    for img_path in tqdm(sorted(glob.glob(f"{FULL_IMG_DIR}/*.jpg"))):
        img = read_image(img_path)
        h, w = img.shape[:2]

        label_path = os.path.join(FULL_LABEL_DIR, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
        gt_boxes = load_gt_yolo(label_path, w, h)  
        gt_labels = [0] * len(gt_boxes) 

        result = get_sliced_prediction(
            img_path,
            detection_model=model,
            slice_height=IMG_SIZE,
            slice_width=IMG_SIZE,
            overlap_height_ratio=SLICE_OVERLAP,
            overlap_width_ratio=SLICE_OVERLAP,
            verbose=False
        )

        pred_boxes = []
        pred_scores = []
        pred_labels = []

        for obj in result.object_prediction_list:
            pred_boxes.append([
                obj.bbox.minx, obj.bbox.miny, obj.bbox.maxx, obj.bbox.maxy
            ])
            pred_scores.append(obj.score.value)
            pred_labels.append(0)  

        preds = [{
            "boxes": torch.tensor(pred_boxes),
            "scores": torch.tensor(pred_scores),
            "labels": torch.tensor(pred_labels),
        }]
        targets = [{
            "boxes": torch.tensor(gt_boxes),
            "labels": torch.tensor(gt_labels),
        }]

        metric.update(preds, targets)

    result = metric.compute()
    print("\nFinal Evaluation on Full Images")
    print(f"mAP@0.5:        {result['map_50']:.4f}")
    print(f"mAP@0.5:0.95:   {result['map']:.4f}")

In [ ]:
model = train_model()

In [ ]:
evaluate_map50('/mnt/c/Study/Python/DroneDetection/runs/yolov8s_auto_augs/weights/best.pt')